In [20]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
import tensorflow as tf
from collections import Counter
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



from tensorflow.python.client import device_lib
device_lib.list_local_devices()
tf.test.is_gpu_available()



urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt", filename="ratings_total.txt")



total_data = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])



total_data['label'] = np.select([total_data.ratings > 3], [1], default=0)



total_data.drop_duplicates(subset=['reviews'], inplace=True)



# 훈련 데이터와 테스트 데이터를 3:1 비율로 분리
train_data, test_data = train_test_split(total_data, test_size = 0.25, random_state = 42)



# 한글과 공백을 제외하고 모두 제거
train_data['reviews'] = train_data['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data['reviews'].replace('', np.nan, inplace=True)



# 중복 제거
test_data.drop_duplicates(subset = ['reviews'], inplace=True) 
# 정규 표현식 수행
test_data['reviews'] = test_data['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") 
# 공백은 Null 값으로 변경
test_data['reviews'].replace('', np.nan, inplace=True) 
# Null 값 제거
test_data = test_data.dropna(how='any') 



from eunjeon import Mecab
mecab = Mecab()



stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']



train_data['tokenized'] = train_data['reviews'].apply(mecab.morphs)
train_data['tokenized'] = train_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])
test_data['tokenized'] = test_data['reviews'].apply(mecab.morphs)
test_data['tokenized'] = test_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])



X_train = train_data['tokenized'].values
y_train = train_data['label'].values
X_test= test_data['tokenized'].values
y_test = test_data['label'].values



# 기계가 텍스트를 숫자로 처리할 수 있도록 훈련 데이터와 텍스트 데이터에 정수 인코딩 수행

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)



# 단어 집합이 생성되는 동시에 각 단어에 고유한 정수 부여
# 등장 횟수가 1회인 단어들은 자연어 처리에서 배제

threshold = 2
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)



# 단어 집합 크기를 토크나이저의 인자로 넘겨주고, 텍스트 시퀀스를 정수 시퀀스로 변환
# 정수 인코딩 과정에서 이보다 큰 숫자가 부여된 단어들은 OOV로 변환

# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)



tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)



# 패딩
# 서로 다른 길이의 샘플들의 길이를 동일하게 맞춰주는 작업

def below_threshold_len(max_len, nested_list):
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))



max_len = 80
below_threshold_len(max_len, X_train)



# 훈련용 리뷰의 99.99가 80이하의 길이를 가지기 때문에, 훈련용 리뷰를 길이 80으로 패딩

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)



# GRU로 네이버 쇼핑 리뷰 감성 분류

from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
    
embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(GRU(hidden_units))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)



loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))



def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = mecab.morphs(new_sentence)
  new_sentence = [word for word in new_sentence if not word in stopwords]
  encoded = tokenizer.texts_to_sequences([new_sentence])
  pad_new = pad_sequences(encoded, maxlen = max_len)

  score = float(loaded_model.predict(pad_new))
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))



sentiment_predict("불친절")



data = pd.read_csv('D:\polo_main.csv')



data

C:\Users\jeonj\AppData\Local\Temp\ipykernel_12492\1071252944.py:43: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['reviews'] = train_data['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
C:\Users\jeonj\AppData\Local\Temp\ipykernel_12492\1071252944.py:51: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['reviews'] = test_data['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


단어 집합(vocabulary)의 크기 : 39741
등장 빈도가 1번 이하인 희귀 단어의 수: 18056
단어 집합에서 희귀 단어의 비율: 45.43418635665937
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 0.7869324583053495
단어 집합의 크기 : 21687
전체 샘플 중 길이가 80 이하인 샘플의 비율: 99.99933302652553
Epoch 1/15
1871/1875 [============================>.] - ETA: 0s - loss: 0.2711 - acc: 0.8988
Epoch 1: val_acc improved from -inf to 0.91270, saving model to best_model.h5
1875/1875 [==============================] - 31s 15ms/step - loss: 0.2711 - acc: 0.8989 - val_loss: 0.2378 - val_acc: 0.9127
Epoch 2/15
1874/1875 [============================>.] - ETA: 0s - loss: 0.2136 - acc: 0.9228
Epoch 2: val_acc improved from 0.91270 to 0.92413, saving model to best_model.h5
1875/1875 [==============================] - 27s 15ms/step - loss: 0.2136 - acc: 0.9228 - val_loss: 0.2114 - val_acc: 0.9241
Epoch 3/15
1873/1875 [============================>.] - ETA: 0s - loss: 0.1973 - acc: 0.9287
Epoch 3: val_acc improved from 0.92413 to 0.92767, saving model to best_model.h5
1875/1875 [==============

,Unnamed: 0,comment,star,date
0,1,163/53 m 사이즈 딱 맞아요! 핏되게 입는 거 좋아하시면 사세요. 라지는 팔 ...,5,2022-06-16
1,2,흰색말고 원색컬러 티셔츠 구매하고싶어서 샀어요!ㅎㅎ 면은 정말 좋고 배송도 나름 빨...,5,2022-07-14
2,3,올이 다 풀려버렸습니다. 박음질 자체가 이상해요. 다른데서 구매한 건 이렇지 않았는...,1,2022-07-06
3,4,"일요일에 주문했는데 목요일 아침에 도착했어요ㅎ 재질도 좋고 약간 루즈핏 원했는데 ,...",5,2022-07-14
4,5,L가 품절이라 혹시나 해서 m 샀는데 좀 타이트하네여 남자 s 사이즈엔 m은 좀 타...,5,2022-07-15
...,...,...,...,...
10833,10834,Xogko ogcocog hogo i,5,2018-07-16
10834,10835,Xypxcochohcocco k kh llh,5,2018-07-16
10835,10836,좋아요~~~~,5,2018-06-18
10836,10837,좋아요 \nM 샀다가 너무 딱 맞아서 \nXl 샀는데 편하게 잘 입네요*^,5,2022-05-13


In [4]:
data['label'] = np.select([data.star > 3], [1], default=0)

In [6]:
data.drop_duplicates(subset=['comment'], inplace=True)

In [10]:
# 중복 제거
data.drop_duplicates(subset = ['comment'], inplace=True) 
# 정규 표현식 수행
data['comment'] = data['comment'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") 
# 공백은 Null 값으로 변경
data['comment'].replace('', np.nan, inplace=True) 
# Null 값 제거
data = data.dropna(how='any') 

C:\Users\jeonj\AppData\Local\Temp\ipykernel_12492\916934235.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  data['comment'] = data['comment'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


In [11]:
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']

In [14]:
data['tokenized'] = data['comment'].apply(mecab.morphs)
data['tokenized'] = data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])

C:\Users\jeonj\AppData\Local\Temp\ipykernel_12492\1258023939.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tokenized'] = data['comment'].apply(mecab.morphs)
C:\Users\jeonj\AppData\Local\Temp\ipykernel_12492\1258023939.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tokenized'] = data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])


In [16]:
X_test= data['tokenized'].values
y_test = data['label'].values

In [17]:
X_test = tokenizer.texts_to_sequences(X_test)

In [18]:
X_test = pad_sequences(X_test, maxlen=max_len)

In [19]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

269/269 [==============================] - 2s 6ms/step - loss: 0.3017 - acc: 0.8928

 테스트 정확도: 0.8928


In [ ]:
# 워드클라우드 함수
def plotting_WordCloud(x):
  # plotting WordCloud
  icon = Image.open('D:\drive/phploeBuh.png').resize((600,600)) # 워드클라우드 모양 변형을 위한 마스크 설정
  mask = np.array(icon)
  wc = WordCloud(font_path = 'D:\drive/NanumGothic.ttf',
                 background_color='white', # 배경색
                 width = 1000, # 넓이
                 height = 1000, # 길이
                 max_words = 100, # 최대 수용 단어 개수
                 max_font_size = 100,
                 mask = mask).generate_from_frequencies(dict(x))
  plt.figure(figsize= (15,10))
  plt.axis('off')
  plt.imshow(wc,interpolation='bilinear')
  return